# 1. Update all data

In [1]:
# %load C:\Users\Varun\Desktop\ads\src\data\get_data.py
import subprocess,os
import pandas as pd
import requests,json
#DIRpath = os.path.join(os.path.dirname(__file__),'../../data/COVID-19') # executing .py file

DIRpath = os.path.join('../data/COVID-19/') #executing IPYNB file
#InputCSVpath_Relative=os.path.join(DIRpath,InputFilepath)
#TargetCSVpath_Relative=os.path.join(DIRpath,TargetFilepath)


def UPDATE_DATABASE_WITH_GIT():
 print('********************')
 print('Updating database...')
 print('********************')
 git_pull= subprocess.Popen('git init & git reset --hard & git pull https://github.com/CSSEGISandData/COVID-19.git',
                            cwd=DIRpath,
                            shell=True,
                            stdout=subprocess.PIPE,
                            stderr=subprocess.PIPE)
 
 
 (out,error)=git_pull.communicate()
 
  
 print('Error:', str(error))
 print('Out:', str(out))
 print('********************')
 print('Database update complete')
 print('********************')

def get_current_data_germany():

    restAPIdata=requests.get('https://services7.arcgis.com/mOBPykOjAyBO2ZKk/arcgis/rest/services/Coronaf%C3%A4lle_in_den_Bundesl%C3%A4ndern/FeatureServer/0/query?where=1%3D1&outFields=*&outSR=4326&f=json')

    json_object=json.loads(restAPIdata.content)
    json_object.keys()
    json_object['features'][0]['attributes'] #to access features and 1st index and attributes


    Myjsonlist=[]

    for items in json_object['features']:
    #print(items ['attributes']['LAN_ew_GEN'])
    #print(items ['attributes']['Fallzahl'])

        Myjsonlist.append(items ['attributes'])

    

    pd_full_list=pd.DataFrame(Myjsonlist)
    #print(Myjsonlist)
    pd_full_list.to_csv(r'C:\Users\Varun\Desktop\ads\data\raw\NPGEO\GER_state_data.csv',sep=';') 
    #print('Number of regions rows:'+str(pd_full_list.shape[0]))
if __name__ == '__main__':
   #get_johns_hopkins()
   UPDATE_DATABASE_WITH_GIT()
   get_current_data_germany()


********************
Updating database...
********************
Error: b'From https://github.com/CSSEGISandData/COVID-19\n * branch              HEAD       -> FETCH_HEAD\n'
Out: b'Reinitialized existing Git repository in C:/Users/Varun/Desktop/ads/data/COVID-19/.git/\nHEAD is now at 6e011224 Update README.md\nUpdating 6e011224..d07a81e2\nFast-forward\n README.md | 7 +++++++\n 1 file changed, 7 insertions(+)\n'
********************
Database update complete
********************


# 2. Process pipeline

In [2]:
# %load C:\Users\Varun\Desktop\ads\src\data\process_JH_data.py
import pandas as pd
import numpy as np
import os

from datetime import datetime


def store_relational_JH_data():  #function defined to convert the COVID data in a relational data set

    #data_path = os.path.join(os.path.dirname(__file__),'../../data/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
    data_path=('../data/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
    pd_raw=pd.read_csv(data_path)

    pd_data_base=pd_raw.rename(columns={'Country/Region':'country',
                      'Province/State':'state'})

    pd_data_base['state']=pd_data_base['state'].fillna('no')

    pd_data_base=pd_data_base.drop(['Lat','Long'],axis=1)


    pd_relational_model=pd_data_base.set_index(['state','country']) \
                                .T                              \
                                .stack(level=[0,1])             \
                                .reset_index()                  \
                                .rename(columns={'level_0':'date',
                                                   0:'confirmed'},
                                                  )
    #TargetCSVpath = os.path.join(os.path.dirname(__file__),'../../data/processed/COVID_relational_confirmed.csv')                                            
    TargetCSVpath = r'../data/processed/COVID_relational_confirmed.csv'
    pd_relational_model['date']=pd_relational_model.date.astype('datetime64[ns]')

    pd_relational_model.to_csv(TargetCSVpath,sep=';',index=False)
    print(' Number of rows stored: '+str(pd_relational_model.shape[0]))
    
if __name__ == '__main__':

    store_relational_JH_data()


 Number of rows stored: 59850


# 3. Slope calculation

# 4. Visual Board